Seeding for reproducibility

In [14]:
import random
random.seed(0)

import numpy as np
np.random.seed(0)

import tensorflow as tf
tf.random.set_seed(0)



Importing the dependencies

In [15]:
import os
import json
from zipfile import ZipFile
from PIL import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tensorflow.keras.preprocessing.image  import ImageDataGenerator
from tensorflow.keras import layers, models

Data Curation


upload the kaggle.json file

In [11]:
!pip install kaggle

In [16]:
kaggle_credentails = json.load(open("kaggle.json"))

In [17]:
os.environ['KAGGLE_USERNAME'] = kaggle_credentails["username"]
os.environ['KAGGLE_KEY'] = kaggle_credentails["key"]

In [18]:
!kaggle datasets download -d abdallahalidev/plantvillage-dataset

Dataset URL: https://www.kaggle.com/datasets/abdallahalidev/plantvillage-dataset
License(s): CC-BY-NC-SA-4.0
('Connection broken: IncompleteRead(464333730 bytes read, 1725052689 more expected)', IncompleteRead(464333730 bytes read, 1725052689 more expected))


In [19]:
!ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [20]:
with ZipFile("plantvillage-dataset.zip" , 'r') as zip_ref:
  zip_ref.extractall()

FileNotFoundError: [Errno 2] No such file or directory: 'plantvillage-dataset.zip'

In [ ]:
print(os.listdir("plantvillage dataset"))

print(len(os.listdir("plantvillage dataset/segmented")))
print(os.listdir("plantvillage dataset/segmented")[:5])

print(len(os.listdir("plantvillage dataset/color")))
print(os.listdir("plantvillage dataset/segmented")[:5])

print(len(os.listdir("plantvillage dataset/grayscale")))
print(os.listdir("plantvillage dataset/grayscale")[:5])


In [ ]:
print(len(os.listdir("plantvillage dataset/color/Grape___healthy")))
print(os.listdir("plantvillage dataset/color/Grape___healthy")[:5])

In [ ]:
base_dir = 'plantvillage dataset/color'

In [ ]:
image_path = 'plantvillage dataset/color/Apple___Cedar_apple_rust/025b2b9a-0ec4-4132-96ac-7f2832d0db4a___FREC_C.Rust 3655.JPG'

img = mpimg.imread(image_path)
print(img.shape)

plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
img_size = 224
batch_size = 32

In [ ]:
#range - 0 to 255 - rescale 0 to 1
data_gen = ImageDataGenerator(
    rescale = 1./255,
    validation_split = 0.2
)

In [ ]:
#Train Generator
train_generator = data_gen.flow_from_directory(
    base_dir,
    target_size = (img_size , img_size),
    batch_size = batch_size,
    subset = 'training',
    class_mode = 'categorical'

)

In [ ]:
#Validation Generator
validation_generator = data_gen.flow_from_directory(
    base_dir,
    target_size = (img_size , img_size),
    batch_size = batch_size,
    subset = 'validation',
    class_mode = 'categorical'

)

In [ ]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape = (img_size , img_size , 3)))
model.add(layers.MaxPooling2D(2,2))


model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))


model.add(layers.Flatten())
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(train_generator.num_classes, activation = 'softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

In [ ]:
#Training the model
history = model.fit (
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    epochs = 10,
    validation_data= validation_generator,
    validation_steps = validation_generator.samples // batch_size
)

In [ ]:
#model evaluation
print("Evaluating Model..")
val_loss , val_accuracy = model.evaluate(validation_generator, steps = validation_generator.samples// batch_size)

print(f"Validation Accuracy:{val_accuracy * 100:.2f}%")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc = 'upper left')
plt.show()

In [ ]:
class_indices = { v: k for k , v in train_generator.class_indices.items()}

In [ ]:
class_indices

In [ ]:
json.dump(class_indices , open('class_indices.json', 'w'))

In [ ]:
#image_path = 'test_potato_early_blight.jpg'
predicted_class_name =predict_image_class(model, image_path , class_indices)

print("Predicted Class Name: " , predicted_class_name)

In [ ]:
model.save('plant_disease_prediction_model.h5')